Importamos las librerias que utilizaremos a lo largo del *notebook*

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import uuid
import math

Como vemos no podemos sacar la información por meses... pasamos a Eurostat

## Eurostat Importaciones/Exportaciones por mes

In [2]:
eurostat = pd.read_csv("eurostat_importaciones_exportaciones/DS-018995_1_Data.csv", encoding = "ISO-8859-1")
eurostat_horto = eurostat.loc[eurostat["PRODUCT_LABEL"]=="VEGETABLES AND FRUIT"]
eurostat_horto = eurostat_horto[eurostat_horto.Value != ":"]
for i, row in eurostat_horto.iterrows():
    eurostat_horto.at[i,'Value'] = row["Value"].replace(" ", "")
eurostat_horto['Value'] = eurostat_horto['Value'].astype(float)
eurostat_horto.head()

,PRODUCT,PRODUCT_LABEL,PARTNER,PARTNER_LABEL,REPORTER,FLOW,PERIOD,INDICATORS,Value,Flag and Footnotes
135840,5,VEGETABLES AND FRUIT,EU28_EXTRA,Extra-EU28 (= 'WORLD' - 'EU28_INTRA'),Spain (incl. Canary Islands 'XB' from 1997),IMPORT,Jan. 2017,VALUE_IN_EUROS,262854626.0,NaN
135841,5,VEGETABLES AND FRUIT,EU28_EXTRA,Extra-EU28 (= 'WORLD' - 'EU28_INTRA'),Spain (incl. Canary Islands 'XB' from 1997),IMPORT,Feb. 2017,VALUE_IN_EUROS,258526740.0,NaN
135842,5,VEGETABLES AND FRUIT,EU28_EXTRA,Extra-EU28 (= 'WORLD' - 'EU28_INTRA'),Spain (incl. Canary Islands 'XB' from 1997),IMPORT,Mar. 2017,VALUE_IN_EUROS,308892347.0,NaN
135843,5,VEGETABLES AND FRUIT,EU28_EXTRA,Extra-EU28 (= 'WORLD' - 'EU28_INTRA'),Spain (incl. Canary Islands 'XB' from 1997),IMPORT,Apr. 2017,VALUE_IN_EUROS,249943804.0,NaN
135844,5,VEGETABLES AND FRUIT,EU28_EXTRA,Extra-EU28 (= 'WORLD' - 'EU28_INTRA'),Spain (incl. Canary Islands 'XB' from 1997),IMPORT,May. 2017,VALUE_IN_EUROS,275080559.0,NaN


In [3]:
eurostat_horto["PARTNER"].unique()

array(['EU28_EXTRA', 'EU28_INTRA', 'AD', 'AE', 'AF', 'AG', 'AL', 'AM',
       'AO', 'AR', 'AT', 'AU', 'AW', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF',
       'BG', 'BH', 'BJ', 'BL', 'BM', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW',
       'BY', 'BZ', 'CA', 'CD', 'CF', 'CG', 'CH', 'CI', 'CL', 'CM', 'CN',
       'CO', 'CR', 'CU', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM',
       'DO', 'DZ', 'EC', 'EE', 'EG', 'EH', 'ET', 'FI', 'FJ', 'FR', 'GA',
       'GB', 'GD', 'GE', 'GH', 'GI', 'GM', 'GN', 'GQ', 'GR', 'GS', 'GT',
       'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN',
       'IQ', 'IR', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KM',
       'KP', 'KR', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR',
       'LT', 'LU', 'LV', 'LY', 'MA', 'MD', 'ME', 'MG', 'MK', 'ML', 'MM',
       'MN', 'MO', 'MR', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', nan,
       'NC', 'NE', 'NG', 'NI', 'NL', 'NO', 'NP', 'NZ', 'OM', 'PA', 'PE',
       'PF', 'PG', 'PH', 'PK', 'PL', 'PS', 'PT', 'PY',

In [4]:
!pip install geopy

In [5]:
# "BL": "Saint Barthélemy" / 'GS': "South Georgia and The South Sandwich Islands"
# 'HK': "Hong Kong" / 'MO': "Macao" / 'TM': "Turkmenistan" / 'KP': "North Korea"
# "BN": "Brunei Darussalam" / "CI": "Côte D'Ivoire" / 'SX': "Sint Maarten"
abreviatura2location = {
    "AD": "Andorra", "AE": "United Arab Emirates", "AF": "Afghanistan", "AG": "Antigua and Barbuda",
    "AL": "Anguilla", "AM": "Armenia", "AO": "Angola", "AR": "Argentina", "AT": "Austria", "AU": "Australia",
    "AW": "Aruba", "AZ": "Azerbaijan", "BA": "Bosnia and Herzegovina", "BB": "Barbados", "BD": "Bangladesh",
    "BE": "Belgium", "BF": "Burkina Faso", "BG": "Bulgaria", "BH": "Bahrain", "BJ": "Benin", 
    "BM": "Bermuda", "BO": "Bolivia", "BR": "Brazil",
    "BS": "Bahamas", "BT": "Bhutan", "BW": "Botswana", "BY": "Belarus", "BZ": "Belize", "CA": "Canada",
    "CD": "Democratic Republic of The Congo", "CF": "Central African Republic", "CG": "Congo",
    "CH": "Switzerland",  "CL": "Chile", "CM": "Cameroon", "CN": "China", 
    "CO": "Colombia", 'CR': "Costa Rica", 'CU': "Cuba", 'CV': "Cape Verde", 'CW': "Curaçao", 'CY': "Cyprus", 
    'CZ': "Czechia", 'DE': "Germany", 'DJ': "Djibouti", 'DK': "Denmark", 'DM': "Dominica", 
    'DO': "Dominican Republic", 'DZ': "Algeria", 'EC': "Ecuador", 'EE': "Estonia", 'EG': "Egypt", 
    'EH': "Western Sahara", 'ET': "Ethiopia", 'FI': "Finland", 'FJ': "Fiji", 'FR': "France", 
    'GA': "Gabon", 'GB': "United Kingdom", 'GD': "Grenada", 'GE': "Georgia", 'GH': "Ghana", 'GI': "Gibraltar", 
    'GM': "Gambia", 'GN': "Guinea", 'GQ': "Equatorial Guinea", 'GR': "Greece", 
    'GT': "Guatemala", 'GW': "Guinea-Bissau", 
    'GY': "Guyana", 'HN': "Honduras", 'HR': "Croatia", 'HT': "Haiti", 'HU': "Hungary", 
    'ID': "Indonesia", 'IE': "Ireland", 'IL': "Israel", 'IN': "Isle of Man", 'IQ': "Iraq", 'IR': "Iran", 
    'IS': "Iceland",  'IT': "Italy",  'JM': "Jamaica",  'JO': "Jordan",  'JP': "Japan",  'KE': "Kenya", 
    'KG': "Kyrgyzstan",  'KH': "Cambodia",  'KM': "Comoros", 'KR': "South Korea", 
    'KW': "Kuwait", 'KY': "Cayman Islands", 'KZ': "Kazakhstan", 'LA': "Laos", 'LB': "Lebanon", 
    'LC': "Saint Lucia", 'LI': "Liechtenstein", 'LK': "Sri Lanka", 'LR': "Liberia", 'LT': "Lithuania", 
    'LU': "Luxembourg", 'LV': "Latvia", 'LY': "Libya", 'MA': "Morocco", 'MD': "Moldova", 'ME': "Montenegro", 
    'MG': "Madagascar", 'MK': "North Macedonia", 'ML': "Mali", 'MM': "Myanmar/Burma",
    'MN': "Mongolia", 'MR': "Mauritania", 'MT': "Malta", 'MU': "Mauritius", 
    'MV': "Maldives", 'MW': "Malawi", 'MX': "Mexico", 'MY': "Malaysia", 'MZ': "Mozambique",
    'NC': "New Caledonia", 'NE': "Niger", 'NG': "Nigeria", 'NI': "Nicaragua", 'NL': "Netherlands", 
    'NO': "Norway", 'NP': "Nepal", 'NZ': "New Zealand", 'OM': "Oman", 'PA': "Panama", 'PE': "Peru",
    'PF': "French Polynesia", 'PG': "Papua New Guinea", 'PH': "Philippines", 'PK': "Pakistan", 
    'PL': "Poland", 'PS': "Palestine", 'PT': "Portugal", 'PY': "Paraguay", 'QA': "Qatar", 'RO': "Romania", 
    'RU': "Russian Federation", 'RW': "Rwanda", 'SA': "Saudi Arabia", 'SC': "Seychelles", 'SD': "Sudan", 
    'SE': "Sweden", 'SG': "Singapore", 'SI': "Slovenia", 'SK': "Slovakia", 'SL': "Sierra Leone", 
    'SM': "San Marino", 'SN': "Senegal", 'SO': "Somalia", 'SR': "Suriname", 'SS': "South Sudan", 
    'ST': "São Tomé and Príncipe", 'SV': "El Salvador", 'SY': "Syria",
    'SZ': "Eswatini", 'TD': "Chad", 'TG': "Togo", 'TH': "Thailand", 'TJ': "Tajikistan", 
    'TL': "Timor-Leste", 'TN': "Tunisia", 'TR': "Turkey", 'TT': "Trinidad and Tobago",
    'TZ': "United Republic of Tanzania", 'UA': "Ukraine", 'UG': "Uganda", 'US': "United States", 
    'UY': "Uruguay", 'UZ': "Uzbekistan", 'VE': "Venezuela", 'VG': "British Virgin Islands", 
    'VN': "Vietnam", 'WS': "Wallis and Futuna", 'YE': "Yemen", 'ZA': "South Africa", 
    'ZM': "Zambia",  'ZW': "Zimbabwe"
}

print(f"Tenemos un total de {len(abreviatura2location)} paises")

Tenemos un total de 181 paises


In [6]:
# https://www.geeksforgeeks.org/how-to-find-longitude-and-latitude-for-a-list-of-regions-or-country-using-python/
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 

In [7]:
country_locations_df = pd.DataFrame(data = {'Country': list(abreviatura2location.values())})

# declare an empty list to store latitude and longitude of values of country column 
latitude, longitude = [], [] 
   
# function to find the coordinate of a given city  
def findGeocode(city): 
    # try and catch: overcome the exception thrown by geolocator using geocodertimedout   
    try:   
        # Specify the user_agent as your app name it should not be none 
        geolocator = Nominatim(user_agent="cajamar") 
        return geolocator.geocode(city) 
    except GeocoderTimedOut: 
        return findGeocode(city)     

# each value from country column will be fetched and sent to function find_geocode    
for pais in (country_locations_df["Country"]): 
    if findGeocode(pais) != None: 
        loc = findGeocode(pais) 
        # coordinates returned from  function is stored into two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not found, alert!
    else: 
        assert False, f"No podemos encontrar las coordenadas para '{pais}'"

country_locations_df["Latitude"] = latitude 
country_locations_df["Longitude"] = longitude 

In [8]:
def truncate(numero, n):
    """
    Dado un numero y n, devuelve el numero con n decimales.
    Ejemplo -> numero = 10.321, n=2 -> 10.32
    """
    if type(numero)==int: return numero
    pos = str(numero).find(".") + n + 1
    return float(str (numero)[:pos])

In [9]:
# http://wiki.gis.com/wiki/index.php/Decimal_degrees
# Podemos guardar únicamente 2 decimales para salvar espacio!

location2coordinates = {}
for index, row in country_locations_df.iterrows():
    location2coordinates[row['Country']] = [truncate(row['Latitude'], 2), truncate(row['Longitude'], 2)]

# location2coordinates => 'Andorra': [42.540, 1.573], 'Angola': [-11.877, 17.569]...

In [11]:
# Dominica no es capaz de cogerlo bien
location2coordinates["Georgia"] = [41.71, 44.78]
location2coordinates["Dominica"] = [15.34, -61.43]

In [12]:
# 'Saint Barthélemy': "San Bartolomé" 
# 'South Georgia and The South Sandwich Islands': "Islas Georgias del Sur y Sandwich del Sur",
# 'Hong Kong': "Hong Kong" / 'Macao': "Macao" / 'Turkmenistan': "Turkmenistán"
# 'North Korea': "Corea del Norte" / 'Brunei Darussalam': "Brunéi"
# "Côte D'Ivoire": "Costa de Marfil" / 
pais2display = {
    'Andorra': "Andorra", 'United Arab Emirates': "Emiratos Árabes Unidos", 'Afghanistan': "Afganistán",
    'Antigua and Barbuda': "Antigua y Barbuda", 'Anguilla': "Anguilla", 'Armenia': "Armenia",
    'Angola': "Angola", 'Argentina': "Argentina", 'Austria': "Austria", 'Australia': "Australia",
    'Aruba': "Aruba", 'Azerbaijan': "Azerbaiyán", 'Bosnia and Herzegovina': "Bosnia y Herzegovina",
    'Barbados': "Barbados", 'Bangladesh': "Bangladesh", 'Belgium': "Bélgica",
    'Burkina Faso': "Burkina Faso", 'Bulgaria': "Bulgaria", 'Bahrain': "Baréin", 'Benin': "Benín",
    'Bermuda': "Bermudas", 
    'Bolivia': "Bolivia", 'Brazil': "Brasil", 'Bahamas': "Bahamas", 'Bhutan': "Bután", 
    'Botswana': "Botsuana", 'Belarus': "Bielorrusia", 'Belize': "Belice", 'Canada': "Canada",
    'Democratic Republic of The Congo': "República Democrática del Congo", 
    'Central African Republic': "República Centroafricana", 'Congo': "Congo", 'Switzerland': "Suiza",
     'Chile': "Chile", 'Cameroon': "Camerún", 'China': "China",
    'Colombia': "Colombia", 'Costa Rica': "Costa Rica", 'Cuba': "Cuba", 'Cape Verde': "Cabo Verde",
    'Curaçao': "Curazao", 'Cyprus': "Chipre", 'Czechia': "Chequia", 'Germany': "Alemania",
    'Djibouti': "Yibuti", 'Denmark': "Dinamarca", 'Dominica': "Dominica", 
    'Dominican Republic': "República Dominicana", 'Algeria': "Argelia", 'Ecuador': "Ecuador",
    'Estonia': "Estonia", 'Egypt': "Egipto", 'Western Sahara': "Sahara Occidental",
    'Ethiopia': "Etiopía", 'Finland': "Finlandia", 'Fiji': "Fiyi", 'France': "Francia",
    'Gabon': "Gabón", 'United Kingdom': "Reino Unido", 'Grenada': "Granada", 'Georgia': "Georgia",
    'Ghana': "Ghana", 'Gibraltar': "Gibraltar", 'Gambia': "Gambia", 'Guinea': "Guinea",
    'Equatorial Guinea': "Guinea Ecuatorial", 'Greece': "Grecia",
    'Guatemala': "Guatemala", 'Guinea-Bissau': "Guinea-Bisáu", 'Guyana': "Guyana",
    'Honduras': "Honduras", 'Croatia': "Croacia", 'Haiti': "Haití", 'Hungary': "Hungría",
    'Indonesia': "Indonesia", 'Ireland': "Irlanda", 'Israel': "Israel", 'Isle of Man': "Isla de Man",
    'Iraq': "Irak", 'Iran': "Irán", 'Iceland': "Islandia", 'Italy': "Italia", 'Jamaica': "Jamaica",
    'Jordan': "Jordania", 'Japan': "Japón", 'Kenya': "Kenia", 'Kyrgyzstan': "Kirguistán",
    'Cambodia': "Camboya", 'Comoros': "Comoras", 
    'South Korea': "Corea del Sur", 'Kuwait': "Kuwait", 'Cayman Islands': "Islas Caimán",
    'Kazakhstan': "Kazajistán", 'Laos': "Laos", 'Lebanon': "Líbano", 'Saint Lucia': "Santa Lucía",
    'Liechtenstein': "Liechtenstein", 'Sri Lanka': "Sri Lanka", 'Liberia': "Liberia",
    'Lithuania': "Lituania", 'Luxembourg': "Luxemburgo", 'Latvia': "Letonia", 'Libya': "Libia",
    'Morocco': "Marruecos", 'Moldova': "Moldavia", 'Montenegro': "Montenegro", 'Madagascar': "Madagascar",
    'North Macedonia': "Macedonia del Norte", 'Mali': "Malí", 'Myanmar/Burma': "Myanmar (Birmania)",
    'Mongolia': "Mongolia", 'Mauritania': "Mauritania", 'Malta': "Malta",
    'Mauritius': "Mauricio", 'Maldives': "Maldivas", 'Malawi': "Malaui", 'Mexico': "México",
    'Malaysia': "Malasia", 'Mozambique': "Mozambique", 'New Caledonia': "Nueva Caledonia",
    'Niger': "Níger", 'Nigeria': "Nigeria", 'Nicaragua': "Nicaragua", 'Netherlands': "Países Bajos",
    'Norway': "Noruega", 'Nepal': "Nepal", 'New Zealand': "Nueva Zelanda", 'Oman': "Omán",
    'Panama': "Panamá", 'Peru': "Perú", 'French Polynesia': "Polinesia Francesa",
    'Papua New Guinea': "Papúa Nueva Guinea", 'Philippines': "Filipinas", 'Pakistan': "Pakistán",
    'Poland': "Polonia", 'Palestine': "Palestina", 'Portugal': "Portugal", 'Paraguay': "Paraguay",
    'Qatar': "Catar", 'Romania': "Rumanía", 'Russian Federation': "Rusia", 'Rwanda': "Ruanda",
    'Saudi Arabia': "Arabia Saudita", 'Seychelles': "Seychelles", 'Sudan': "Sudán", 'Sweden': "Suecia",
    'Singapore': "Singapur", 'Slovenia': "Eslovenia", 'Slovakia': "Eslovaquia", 
    'Sierra Leone': "Sierra Leona", 'San Marino': "San Marino", 'Senegal': "Senegal", 'Somalia': "Somalia",
    'Suriname': "Surinam", 'South Sudan': "Sudán del Sur", 'São Tomé and Príncipe': "Santo Tomé y Príncipe",
    'El Salvador': "El Salvador", 'Sint Maarten': "Sint Maarten", 'Syria': "Siria", 'Eswatini': "Esuatini",
    'Chad': "Chad", 'Togo': "Togo", 'Thailand': "Tailandia", 'Tajikistan': "Tayikistán",
    'Timor-Leste': "Timor Oriental", 'Tunisia': "Túnez",
    'Turkey': "Turquía", 'Trinidad and Tobago': "Trinidad y Tobago", 'United Republic of Tanzania': "Tanzania",
    'Ukraine': "Ucrania", 'Uganda': "Uganda", 'United States': "Estados Unidos", 'Uruguay': "Uruguay",
    'Uzbekistan': "Uzbekistán", 'Venezuela': "Venezuela", 'British Virgin Islands': "Islas Vírgenes Británicas",
    'Vietnam': "Vietnam", 'Wallis and Futuna': "Wallis y Futuna", 'Yemen': "Yemen", 'South Africa': "Sudáfrica",
    'Zambia': "Zambia", 'Zimbabwe': "Zimbabue", "Spain": "España"
}

In [13]:
eurostat_por_pais = eurostat_horto.groupby("PARTNER")

In [14]:
eurostat_horto.PERIOD.unique()

array(['Jan. 2017', 'Feb. 2017', 'Mar. 2017', 'Apr. 2017', 'May. 2017',
       'Jun. 2017', 'Jul. 2017', 'Aug. 2017', 'Sep. 2017', 'Oct. 2017',
       'Nov. 2017', 'Dec. 2017', 'Jan. 2018', 'Feb. 2018', 'Mar. 2018',
       'Apr. 2018', 'May. 2018', 'Jun. 2018', 'Jul. 2018', 'Aug. 2018',
       'Sep. 2018', 'Oct. 2018', 'Nov. 2018', 'Dec. 2018', 'Jan. 2019',
       'Feb. 2019', 'Mar. 2019', 'Apr. 2019', 'May. 2019', 'Jun. 2019',
       'Jul. 2019', 'Aug. 2019', 'Sep. 2019', 'Oct. 2019', 'Nov. 2019',
       'Dec. 2019', 'Jan. 2020', 'Feb. 2020', 'Mar. 2020', 'May. 2020',
       'Jun. 2020', 'Jul. 2020', 'Aug. 2020', 'Sep. 2020', 'Oct. 2020',
       'Dec. 2020', 'Apr. 2020', 'Nov. 2020'], dtype=object)

In [15]:
months_abreviatura = {
    "Jan.":1, "Feb.":2, "Mar.":3, "Apr.":4, "May.":5, "Jun.":6, 
    "Jul.":7, "Aug.":8, "Sep.":9, "Oct.":10, "Nov.":11, "Dec.":12
}

analized_years = [2018, 2019, 2020]

In [16]:
paises = []
res = []
importaciones_sum, exportaciones_sum = [], []
importaciones_marzo_2018, importaciones_marzo_2019, importaciones_marzo_2020 = [], [], []
importaciones_abril_2018, importaciones_abril_2019, importaciones_abril_2020 = [], [], []

importaciones_excepcion_2018, importaciones_excepcion_2019, importaciones_excepcion_2020 = [], [], []
exportaciones_excepcion_2018, exportaciones_excepcion_2019, exportaciones_excepcion_2020 = [], [], []

for pais_short, group in eurostat_por_pais:
    if pais_short in abreviatura2location:
        pais = abreviatura2location[pais_short]
        
        for mes_abreviado in months_abreviatura:
            for year_analizado in analized_years:
                
                periodo = group[group['PERIOD'] == f"{mes_abreviado} {year_analizado}"]
                importaciones_periodo = int(periodo.loc[periodo['FLOW']=='IMPORT'].Value.sum())
                exportaciones_periodo = int(periodo.loc[periodo['FLOW']=='EXPORT'].Value.sum())
                if (importaciones_periodo + exportaciones_periodo) > 0:
                    res.append({
                       "type":"Feature",
                        "properties":{
                            "id": uuid.uuid4().hex,
                            "countryISO": pais_short,
                            "Month": months_abreviatura[mes_abreviado],
                            "Year": year_analizado,
                            "Imports": importaciones_periodo,
                            "Exports": exportaciones_periodo,
                            "Reporter": pais2display[pais]
                        },
                        "geometry":{
                            "type": "Point",
                            "coordinates": location2coordinates[pais][::-1]
                        }
                    })

In [17]:
res = {
  "type": "FeatureCollection",
  "features": res
}

with open('imports_exports_full.min.geojson', 'w') as fp:
    json.dump(res, fp, separators=(',', ':'))

## Choropleth covid

In [29]:
!pip install geojson

In [30]:
import geojson
# CNTR_RG_01M_2020_4326 - CNTR_RG_60M_2020_4326
with open("fronteras/CNTR_RG_20M_2020_4326.geojson") as f:
    choropleth_locations = geojson.load(f)

In [31]:
country_position = {}
for index in range(len(choropleth_locations['features'])):
    country_id = choropleth_locations['features'][index]["properties"]["NAME_ENGL"]
    if country_id in country_position:
        assert False, "Repeated countries?!"
    country_position[country_id] = index
print(country_position)

{'Argentina': 0, 'American Samoa': 1, 'Austria': 2, 'Antarctica': 3, 'Andorra': 4, 'United Arab Emirates': 5, 'Afghanistan': 6, 'Antigua and Barbuda': 7, 'Anguilla': 8, 'Albania': 9, 'Armenia': 10, 'Angola': 11, 'Australia': 12, 'Aruba': 13, 'Azerbaijan': 14, 'Bosnia and Herzegovina': 15, 'Barbados': 16, 'Bangladesh': 17, 'Belgium': 18, 'Burkina Faso': 19, 'Bulgaria': 20, 'Bahrain': 21, 'Burundi': 22, 'Benin': 23, 'Saint Barthélemy': 24, 'Bermuda': 25, 'Brunei': 26, 'Bolivia': 27, 'Bonaire, Sint Eustatius and Saba': 28, 'Brazil': 29, 'China': 30, 'Bahamas': 31, 'Bhutan': 32, 'Bouvet Island': 33, 'Botswana': 34, 'Belarus': 35, 'Belize': 36, 'Colombia': 37, 'Clipperton Island': 38, 'Costa Rica': 39, 'Cuba': 40, 'Cape Verde': 41, 'Curaçao': 42, 'Christmas Island': 43, 'Cyprus': 44, 'Czechia': 45, 'Cameroon': 46, 'Canada': 47, 'Cocos (Keeling) Islands': 48, 'Democratic Republic of The Congo': 49, 'Central African Republic': 50, 'Congo': 51, 'Switzerland': 52, 'Côte D’Ivoire': 53, 'Greenlan

In [32]:
choropleth_locations['features'][country_position["Andorra"]]["geometry"]

{"coordinates": [[[1.7258, 42.5044], [1.46852, 42.44399], [1.44257, 42.60367], [1.50547, 42.63238], [1.52727, 42.64232], [1.56476, 42.63238], [1.7861, 42.57366], [1.7258, 42.5044]]], "type": "Polygon"}

In [33]:
print(f"Info de -> {abreviatura2location}")

Info de -> {'AD': 'Andorra', 'AE': 'United Arab Emirates', 'AF': 'Afghanistan', 'AG': 'Antigua and Barbuda', 'AL': 'Anguilla', 'AM': 'Armenia', 'AO': 'Angola', 'AR': 'Argentina', 'AT': 'Austria', 'AU': 'Australia', 'AW': 'Aruba', 'AZ': 'Azerbaijan', 'BA': 'Bosnia and Herzegovina', 'BB': 'Barbados', 'BD': 'Bangladesh', 'BE': 'Belgium', 'BF': 'Burkina Faso', 'BG': 'Bulgaria', 'BH': 'Bahrain', 'BJ': 'Benin', 'BM': 'Bermuda', 'BO': 'Bolivia', 'BR': 'Brazil', 'BS': 'Bahamas', 'BT': 'Bhutan', 'BW': 'Botswana', 'BY': 'Belarus', 'BZ': 'Belize', 'CA': 'Canada', 'CD': 'Democratic Republic of The Congo', 'CF': 'Central African Republic', 'CG': 'Congo', 'CH': 'Switzerland', 'CL': 'Chile', 'CM': 'Cameroon', 'CN': 'China', 'CO': 'Colombia', 'CR': 'Costa Rica', 'CU': 'Cuba', 'CV': 'Cape Verde', 'CW': 'Curaçao', 'CY': 'Cyprus', 'CZ': 'Czechia', 'DE': 'Germany', 'DJ': 'Djibouti', 'DK': 'Denmark', 'DM': 'Dominica', 'DO': 'Dominican Republic', 'DZ': 'Algeria', 'EC': 'Ecuador', 'EE': 'Estonia', 'EG': 'Egy

In [139]:
covid_full = pd.read_csv("owid/covid-data.csv")
covid_full.date = pd.to_datetime(covid_full.date)
covid_full["day"] = covid_full.date.dt.day
covid_full["month"] = covid_full.date.dt.month
covid_full["year"] = covid_full.date.dt.year
covid_full.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,day,month,year
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,24,2,2020
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,25,2,2020
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,26,2,2020
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,27,2,2020
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,28,2,2020


In [99]:
fix_pais = {"Russian Federation":"Russia", "Democratic Republic of The Congo":"Congo"}
paises_analisis_covid = {**abreviatura2location, **{"ES": "Spain"}}
for pais_id in paises_analisis_covid:
    
    pais = paises_analisis_covid[pais_id]
    
    if pais in fix_pais: pais = fix_pais[pais]
        
    pais_covid_info = covid_full.loc[(covid_full["location"]==pais.replace(" ", "_"))]
    if len(pais_covid_info)==0:
        pais_covid_info = covid_full.loc[covid_full["location"]==pais.replace(" ", " ")]
        if len(pais_covid_info)==0:
            print(pais)

Aruba
Curaçao
Western Sahara
Myanmar/Burma
New Caledonia
French Polynesia
São Tomé and Príncipe
Timor-Leste
United Republic of Tanzania
British Virgin Islands
Wallis and Futuna


In [108]:
fix_pais = {"Russian Federation":"Russia", "Democratic Republic of The Congo":"Congo"}

res = []

max_cases = -1

paises_analisis_covid = {**abreviatura2location, **{"ES": "Spain"}}

for pais_id in paises_analisis_covid:
    pais = paises_analisis_covid[pais_id]
    
    if pais in fix_pais: 
        paisdf = fix_pais[pais]
    else:
        paisdf = pais
        
    pais_covid_info = covid_full.loc[(covid_full["location"]==paisdf.replace(" ", "_"))]
    if len(pais_covid_info)==0:
        pais_covid_info = covid_full.loc[covid_full["location"]==paisdf.replace(" ", " ")]
        if len(pais_covid_info)==0:
            print(f"{paisdf} Skipped")
            continue
    
    for mes in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
    
        info_mes_2020 = pais_covid_info.loc[(pais_covid_info["month"]==mes) & (pais_covid_info["year"]==2020)]
        
        if int(info_mes_2020["new_cases"].sum()) > 0:
        
            res.append({
                "type":"Feature",
                "properties":{
                    "id": uuid.uuid4().hex,
                    #"countryISO": pais_id,
                    "Month": mes,
                    "Year": 2020,
                    "cases": int(info_mes_2020["new_cases"].sum()),
                    "deaths": int(info_mes_2020["new_deaths"].sum()),
                    "Reporter": pais2display[pais]
                },
                "geometry":{
                    **choropleth_locations['features'][country_position[pais]]["geometry"]
                }
            })

            max_cases = max_cases if max_cases > int(info_mes_2020["new_cases"].sum()) else int(info_mes_2020["new_cases"].sum())

Aruba Skipped
Curaçao Skipped
Western Sahara Skipped
Myanmar/Burma Skipped
New Caledonia Skipped
French Polynesia Skipped
São Tomé and Príncipe Skipped
Timor-Leste Skipped
United Republic of Tanzania Skipped
British Virgin Islands Skipped
Wallis and Futuna Skipped


In [110]:
res = {
  "type": "FeatureCollection",
  "features": res
}

with open('covid_choropleth_20m_min_full.geojson', 'w') as fp:
    json.dump(res, fp, separators=(',', ':'))